In [ ]:
import numpy as np
import pandas as pd
import cv2 
from google.colab.patches import cv2_imshow
from time import time
import dlib
import matplotlib.pyplot as pyt
# creating a function which we need to extract the indexes of the traingles from landmark array
def extract_index(nparray):
  index = None
  for num in nparray[0]:
    index = num
    break
  return index
  

cp = cv2.VideoCapture(0)

# loading modles to detect facelandmarks
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

triangle_indexes = []

# face 1
# Creating 68 face landmarks for source face
faces = detector(img1_gray)
for face in faces:
  landmarks = predictor(img1_gray,face)
  landmarks_points = []

  for n in range(0,68):
    x = landmarks.part(n).x
    y = landmarks.part(n).y
    landmarks_points.append((x,y))

  points = np.array(landmarks_points, np.int32)
  
  convexhull = cv2.convexHull(points)

  cv2.fillConvexPoly(mask, convexhull, 255)

  face_image_1 = cv2.bitwise_and(img1, img1,mask=mask)

  # Delaunay triangulation
  faces = cv2.boundingRect(convexhull)
  subdiv = cv2.Subdiv2D(faces)
  subdiv.insert(landmarks_points)
  triangles = subdiv.getTriangleList()
  triangles = np.array(triangles, dtype=np.int32)

  indexes_triangles = []
  for t in triangles:
    pt1 = (t[0],t[1])
    pt2 = (t[2],t[3])
    pt3 = (t[4],t[5])

    index_pt1 = np.where((points == pt1).all(axis=1))
    index_pt1 = extract_index(index_pt1)

    index_pt2 = np.where((points == pt2).all(axis=1))
    index_pt2 = extract_index(index_pt2)

    index_pt3 = np.where((points == pt3).all(axis=1))
    index_pt3 = extract_index(index_pt3)

    if index_pt1 is not None and index_pt2 is not None and index_pt3 is not None:
      triangle = [index_pt1, index_pt2,index_pt3]
      indexes_triangles.append(triangle)

    # face 2

while True:
  (_, img2) = cp.read()
  #img2_gray = cv2.imread(img2,0)
  img2_gray = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
  img2_new_face = np.zeros_like(img2)
  
  faces2 = detector(img2_gray)
  for face in faces2:
    landmarks = predictor(img2_gray, face)
    landmarks_points2 = []
    for face in range(0,68):
      x = landmark.part(n).x
      y = landmarks.part(n).y
      landmarks_points2.append((x,y))

    points2 = np.array(landmarks_points2, np.int32)
    convexbull2 = cv2.convexHull(points2)

  lines_space_mask = np.zeros_like(img_gray)
  line_space_new_face = np.zeros_like(img2)

  # Triangulation of both faces
  for triangle_index in indexes_triangles:

    # Triangulation of source face 1
    tr1_pt1 = landmarks_points[triangle_index[0]]
    tr1_pt2 = landmarks_points[triangle_index[1]]
    tr1_pt3 = landmarks_points[triangle_index[2]]
    triangle1 = np.array([tr1_pt1, tr1_pt2, tr1_pt3], np.int32)

    rect1 = cv2.boundingRect(triangle1)
    (x, y, w, h) = rect1
    cropped_triangle = img[y: y + h, x: x + w]
    cropped_tr1_mask = np.zeros((h, w), np.uint8)

    points = np.array([[tr1_pt1[0] - x, tr1_pt1[1] - y],
                       [tr1_pt2[0] - x, tr1_pt2[1] - y],
                       [tr1_pt3[0] - x, tr1_pt3[1] - y]], np.int32)

    cv2.fillConvexPoly(cropped_tr1_mask, points, 255)

    # Trianulation of destination face 2
 
    tr2_pt1 = landmarks_points2[triangle_index[0]]
    tr2_pt2 = landmarks_points2[triangle_index[1]]
    tr2_pt3 = landmarks_points2[triangle_index[2]]
    triangle2 = np.array([tr2_pt1, tr2_pt2, tr2_pt3], np.int32)

    rect2 = cv2.boundingRect(triangle2)
    (x, y, w, h) = rect2

    cropped_tr2_mask = np.zeros((h, w), np.uint8)

    points2 = np.array([[tr2_pt1[0] - x, tr2_pt1[1] - y],
                        [tr2_pt2[0] - x, tr2_pt2[1] - y],
                        [tr2_pt3[0] - x, tr2_pt3[1] - y]], np.int32)

    cv2.fillConvexPoly(cropped_tr2_mask, points2, 255)

    # Warp triangles
    points = np.float32(points)
    points2 = np.float32(points2)
    M = cv2.getAffineTransform(points, points2)
    warped_triangle = cv2.warpAffine(cropped_triangle, M, (w, h))
    warped_triangle = cv2.bitwise_and(warped_triangle, warped_triangle, mask=cropped_tr2_mask)

    # Reconstructing destination face
    img2_new_face_rect_area = img2_new_face[y: y + h, x: x + w]
    img2_new_face_rect_area_gray = cv2.cvtColor(img2_new_face_rect_area, cv2.COLOR_BGR2GRAY)
    _, mask_triangles_designed = cv2.threshold(img2_new_face_rect_area_gray, 1, 255, cv2.THRESH_BINARY_INV)
    warped_triangle = cv2.bitwise_and(warped_triangle, warped_triangle, mask=mask_triangles_designed)

    img2_new_face_rect_area = cv2.add(img2_new_face_rect_area, warped_triangle)
    img2_new_face[y: y + h, x: x + w] = img2_new_face_rect_area

  # Face swapped (putting 1st face into 2nd face)
  img2_face_mask = np.zeros_like(img2_gray)
  img2_head_mask = cv2.fillConvexPoly(img2_face_mask, convexhull2, 255)
  img2_face_mask = cv2.bitwise_not(img2_head_mask)


  img2_head_noface = cv2.bitwise_and(img2, img2, mask=img2_face_mask)
  result = cv2.add(img2_head_noface, img2_new_face)

  (x, y, w, h) = cv2.boundingRect(convexhull2)
  center_face2 = (int((x + x + w) / 2), int((y + y + h) / 2))

  seamlessclone = cv2.seamlessClone(result, img2, img2_head_mask, center_face2, cv2.MIXED_CLONE)

  cv2.imshow("img2", img2)
  cv2.imshow("clone", seamlessclone)
  cv2.imshow("result", result)

  key = cv2.waitKey(1)
  if key == 27:
      break

cp.release()
cv2.destroyAllWindows()